# PyCity Schools Analysis

Observed Trends:
* 1 Large schools have a smaller overall passing rate (76.36%) than small and medium schools (94.82% and 95.20%, separately). 
* 2 Schools with higher spending budget per student has a lower overall passing rate.
* 3 Charter schools has a higher overall passing rate than District schools.

In [1]:
# Import dependencies
import pandas as pd
import numpy as np

# Using pd to read files as dataframes
students_file = "./raw_data/students_complete.csv"
schools_file = "./raw_data/schools_complete.csv"
students_df = pd.read_csv(students_file)
schools_df = pd.read_csv(schools_file)

## District Summary

In [2]:
# Merge two dataframes on the school name column
df = pd.merge(students_df, schools_df, left_on = "school", right_on = "name", how="left", suffixes=("_students", "_schools"))
df.head()

# Calculations
total_schools = len(schools_df["School ID"].unique())
total_students = len(students_df["Student ID"].unique())
total_budget = schools_df["budget"].sum()
average_math_score = students_df["math_score"].mean()
average_reading_score = students_df["reading_score"].mean()
percent_passing_math = 100 * len(students_df[students_df["math_score"] >= 70]) / total_students
percent_passing_reading = 100 * len(students_df[students_df["reading_score"] >= 70]) / total_students
overall_passing = (percent_passing_math + percent_passing_reading) / 2

# Generate a dataframe
district_summary = pd.DataFrame(
{
    "Total Schools" : [total_schools],
    "Total Students" : [total_students],
    "Total Budget" : [total_budget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "% Passing Math" : [percent_passing_math],
    "% Passing Reading" : [percent_passing_reading],
    "% Overall Passing Rate" : [overall_passing]
})

# Arrange columns in dataframe
columns = ["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
district_summary = district_summary[columns]
district_summary = district_summary.round(2)
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.99,81.88,74.98,85.81,80.39


## School Summary

In [3]:
# Create a school groupby object
school_group = df.groupby("school")
school_group.min().head()

# Create a data series of students in each school
students_in_schools = df["school"].value_counts()

# Calculate percent passing math and percent passing reading, and overall passing rate
math_pass_df = df[df["math_score"] >= 70]
percent_passing_math = 100 * math_pass_df["school"].value_counts() / students_in_schools
reading_pass_df = df[df["reading_score"] >= 70]
percent_passing_reading = 100 * reading_pass_df["school"].value_counts() / students_in_schools
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

# Generate a dataframe based on data series
school_summary = pd.DataFrame(
{
    "School Type" : school_group["type"].max(),
    "Total Students" : students_in_schools,
    "Total School Budget" : school_group["budget"].mean(),
    "Per Student Budget" : (school_group["budget"].mean()) / students_in_schools,
    "Average Math Score" : (school_group["math_score"].sum()) / students_in_schools,
    "Average Reading Score" : (school_group["reading_score"].sum()) / students_in_schools,
    "% Passing Math" : percent_passing_math,
    "% Passing Reading" : percent_passing_reading,
    "% Overall Passing Rate" : overall_passing_rate
})

# Format budget into $ and decimals, arrange column display for dataframe
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:.2f}".format)
columns = ["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
school_summary = school_summary[columns]
school_summary = school_summary.round(2)
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27


## Top Performing Schools (By Passing Rate)

In [4]:
# Sort schools by overall passing rate, from high to low
school_summary.sort_values("% Overall Passing Rate", ascending = False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.20


## Bottom Performing Schools (By Passing Rate)

In [5]:
# Sort schools by overall passing rate, from low to high
school_summary.sort_values("% Overall Passing Rate", ascending = True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80


## Math Scores by Grade

In [6]:
# Define a function to get the average math score for a certain grade in schools
def math(grade):
    return df[df["grade"] == grade].groupby("school")["math_score"].mean()

# Generate a dataframe based on math scores by grade in schools
math_by_grade = pd.DataFrame(
{
    "9th" : math("9th"),
    "10th" : math("10th"),
    "11th" : math("11th"),
    "12th" : math("12th")
})

# Arrange columns of dataframe
columns = ["9th", "10th", "11th", "12th"]
math_by_grade = math_by_grade[columns]
math_by_grade = math_by_grade.round(2)
math_by_grade.head()

,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


## Reading Score by Grade 

In [7]:
# Define a function to get the average reading score for a certain grade in schools
def reading(grade):
    return df[df["grade"] == grade].groupby("school")["reading_score"].mean()

# Generate a dataframe based on reading scores by grade in schools
reading_by_grade = pd.DataFrame(
{
    "9th" : reading("9th"),
    "10th" : reading("10th"),
    "11th" : reading("11th"),
    "12th" : reading("12th")
})

# Arrange columns of dataframe
columns = ["9th", "10th", "11th", "12th"]
reading_by_grade = reading_by_grade[columns]
reading_by_grade = reading_by_grade.round(2)
reading_by_grade.head()

,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


## Scores by School Spending

In [8]:
# Cut values of per student budget, and place them into bins
bins = [0, 585, 615, 645, 675]
labels = ["<$585", "$585-615", "$615-645", "$645-675"]

# Calcute the spending per student again(their values in school_summary are formatted, so I redo the calculations)
spending_per_student = (school_group["budget"].mean()) / students_in_schools

# Creating a group based off of the bins, and use groupby on the dataframe
school_summary["Spending Ranges (Per Student)"] = pd.cut(spending_per_student, bins, labels = labels)
scores_by_spending = school_summary.groupby("Spending Ranges (Per Student)").mean()

# Arrange columns of dataframe
columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
scores_by_spending = scores_by_spending[columns]
scores_by_spending = scores_by_spending.round(2)
scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46,96.61,95.04
$585-615,83.60,83.88,94.23,95.90,95.06
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


## Scores by School Size

In [9]:
# Cut values of total students and place them into bins
bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Creating a group based off of the bins, and use groupby on the dataframe
school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins, labels = labels)
scores_by_size = school_summary.groupby("School Size").mean()

# Arrange columns of dataframe
columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
scores_by_size = scores_by_size[columns]
scores_by_size = scores_by_size.round(2)
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.87,93.60,96.79,95.20
Large (2000-5000),77.74,81.34,69.96,82.77,76.36


## Scores by School Type

In [12]:
# Using groupby, get summary for each school type
scores_by_type = pd.DataFrame(school_summary.groupby("School Type").mean())

# Arrange columns of dataframe and format
columns = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
scores_by_type = scores_by_type[columns]
scores_by_type = scores_by_type.round(2)
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
